In [ ]:
def check_p_sq_rank (K_cyc, p):
    nr = 0
    for l in K_cyc:
        c = int(l)
        if c%(p**2) == 0:
            nr += 1
    return nr

print("The following line should read [1, 1, 2, 1]")
[check_p_sq_rank(['3', '27', '3'], 3),
    check_p_sq_rank(['3', '45', '5'], 3),
    check_p_sq_rank(['9', '27'], 3),
    check_p_sq_rank(['5', '25', '5'], 5)]

In [6]:
def p_part(cycs):
    
    for cyc in cycs:
        
        for i in range(len(cyc)):
            e=0
            c = cyc[i]
            done = False
            while not done:
                if c%(3**(e+1)) == 0:
                    e += 1
                else:
                    cyc[i] = 3**e
                    done = True
        cyc[:] = [x for x in cyc if x != 1]
    return cycs
p_part([[9, 3], [18, 6], [18, 6], [3, 3, 3]])
p_part([[6, 6, 3], [18, 6], [54, 18], [18, 6, 2, 2]])


[[3, 3, 3], [9, 3], [27, 9], [9, 3]]

In [7]:
from collections import Counter
# Convert lists to tuples (which are hashable) before using Counter
counter = Counter([tuple(lst) for lst in [[1,2], [1,2]]])
counter.items()


dict_items([((1, 2), 2)])

In [8]:
from collections import Counter

def IPAD(K_cyc, cycs):
    # Convert lists to tuples (which are hashable) before using Counter
    counter = Counter([tuple(cyc) for cyc in cycs])
    IPAD = [[list(k), v] for k, v in counter.items()]  # Convert back to lists for output
    return [K_cyc, IPAD]

IPAD([3,3], [[3,9], [3,9], [3,9], [3,3,3]])

[[3, 3], [[[3, 9], 3], [[3, 3, 3], 1]]]

In [ ]:
import re, subprocess, json
import os

executable_path = './extension-class-groups/build/ext-class-groups'
#executable_path = '/Users/eric/Documents/Matematik/Massey-pari/massey'
p = "3"

def run_once(command, timeout=600):
    try:
        r = subprocess.run(command, capture_output=True, text=True,
                           timeout=timeout, check=False, close_fds=True)
        return r.stdout, r.stderr, r.returncode
    except subprocess.TimeoutExpired:
        return None, None, 'TIMEOUT'

infile  = "p_3_all_cyc_[3^m,3^n]"
outfile = "IPADs.json"

# Load existing JSON file if it exists
existing_data = []
existing_d_values = set()

if os.path.exists(outfile):
    print(f"Loading existing data from {outfile}...")
    try:
        with open(outfile, 'r') as f:
            existing_data = json.load(f)
        existing_d_values = {entry["D"] for entry in existing_data}
        print(f"Loaded {len(existing_data)} existing entries")
        print(f"Found {len(existing_d_values)} unique D values already in file")
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Warning: Could not load existing file: {e}")
        print("Starting with empty data")
        existing_data = []
        existing_d_values = set()
else:
    print(f"Output file {outfile} does not exist. Starting fresh.")

# Process new entries
new_entries = []
skipped_existing = 0
processed_new = 0

with open(infile, "r") as f:
    for i, line in enumerate(f):
        if i%100 == 0:
            print(i)
        # if i > 100:
        #     break
        D = line.strip()[1:]
        
        # Check if D already exists in the output file
        if D in existing_d_values:
            skipped_existing += 1
            if skipped_existing % 1000 == 0:
                print(f"Skipped {skipped_existing} existing D values...")
            continue
        
        pol = "s^2+" + D
        #print(f"{i} : D: {D}", flush=True)

        stdout, stderr, rc = run_once([executable_path, p, pol], timeout=10)
        if rc == 'TIMEOUT' or stdout is None:
            print(f"Skipped (timeout/runner issue): D={D}")
            continue

        full = stdout
        
        full = stdout
        c = full.find('"C"');  d = full.find('"C"',  c + 3)
        a = full.find('""'); b = full.find('""', a + 2)

        if a == -1 or b == -1 or c == -1 or d == -1:
            print(f"Missing markers, skipping D={D}")
            continue
        
        output_K_cyc = full[c+3:d]
        output = full[a+2:b]

        # Get the p-part of the extension cyc
        # print(output)
        K_cyc = p_part([eval(output_K_cyc)])
        cycs = p_part(eval(output))

        # Create JSON object
        json_entry = {
            "D": D,
            "IPAD": IPAD(K_cyc[0], cycs)
        }
        new_entries.append(json_entry)
        processed_new += 1

# Combine existing and new entries
print(f"\nSummary:")
print(f"  - Existing entries: {len(existing_data)}")
print(f"  - New entries processed: {processed_new}")
print(f"  - Skipped (already existed): {skipped_existing}")
print(f"  - Total entries to write: {len(existing_data) + len(new_entries)}")

# Write combined data to file
print(f"\nWriting to {outfile}...")
all_data = existing_data + new_entries
with open(outfile, 'w') as out:
    json.dump(all_data, out, indent=2, ensure_ascii=False)

print(f"Successfully wrote {len(all_data)} entries to {outfile}")

Loading existing data from gap-files/IPADs-3.json...
Loaded 461923 existing entries
Found 461923 unique D values already in file
0
100
200
300
400
500
600
700
800
900
Skipped 1000 existing D values...
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
Skipped 2000 existing D values...
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
Skipped 3000 existing D values...
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
Skipped 4000 existing D values...
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
Skipped 5000 existing D values...
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
Skipped 6000 existing D values...
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
Skipped 7000 existing D values...
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
Skipped 8000 existing D values...
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
Skipped 9000 existing D values...
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
Skipped 10000 existing D values...
10000
10100
10200
10300
10400
10500
10600


In [ ]:
import os, json
# Check which D values exist in input file but not in output JSON file
infile  = "p_3_all_cyc_[3^m,3^n]"
outfile = "IPADs.json"

print("Reading D values from input file...")
input_d_values = set()
empty_lines = 0

with open(infile, "r") as f:
    for line_num, line in enumerate(f, 1):
        stripped = line.strip()
        
        # Skip empty lines
        if not stripped:
            empty_lines += 1
            continue
        
        # Check if line has at least 2 characters (need at least "-" + one digit)
        if len(stripped) < 2:
            empty_lines += 1
            continue
        
        d_val = stripped[1:]
        
        # Skip if D is empty after removing first character
        if not d_val:
            empty_lines += 1
            continue
        
        input_d_values.add(d_val)

print(f"  - Total lines in input file: {line_num}")
print(f"  - Empty/invalid lines: {empty_lines}")
print(f"  - Unique D values in input: {len(input_d_values)}")

print("\nReading D values from output JSON file...")
output_d_values = set()

if os.path.exists(outfile):
    with open(outfile, 'r') as f:
        data = json.load(f)
        output_d_values = {entry["D"] for entry in data}
    print(f"  - Total entries in output file: {len(output_d_values)}")
else:
    print(f"  - Output file does not exist")

# Find missing D values
missing_d_values = input_d_values - output_d_values

print(f"\n{'=' * 80}")
print("Comparison Results:")
print(f"{'=' * 80}")
print(f"  - D values in input file: {len(input_d_values)}")
print(f"  - D values in output file: {len(output_d_values)}")
print(f"  - Missing D values: {len(missing_d_values)}")

if missing_d_values:
    print(f"\n⚠️  Found {len(missing_d_values)} D values in input but not in output:")
    sorted_missing = sorted(missing_d_values)
    print(f"  Missing D values: {sorted_missing}")
    
    # Also show which D values are in output but not in input (should be 0 or very few)
    extra_d_values = output_d_values - input_d_values
    if extra_d_values:
        print(f"\n  Note: {len(extra_d_values)} D values in output but not in input:")
        print(f"  Extra D values: {sorted(extra_d_values)[:10]}")
        if len(extra_d_values) > 10:
            print(f"  ... and {len(extra_d_values) - 10} more")
else:
    print(f"\n✓ All D values from input file are present in output file!")
    
    # Still check for extras
    extra_d_values = output_d_values - input_d_values
    if extra_d_values:
        print(f"\n  Note: {len(extra_d_values)} D values in output but not in input:")
        print(f"  Extra D values: {sorted(extra_d_values)}")

Reading D values from input file...
  - Total lines in input file: 461925
  - Empty/invalid lines: 0
  - Unique D values in input: 461925

Reading D values from output JSON file...
  - Total entries in output file: 461923

Comparison Results:
  - D values in input file: 461925
  - D values in output file: 461923
  - Missing D values: 2

⚠️  Found 2 D values in input but not in output:
  Missing D values: ['12132404', '3338296']


In [ ]:
import re, subprocess, json
import os

executable_path = './extension-class-groups/build/ext-class-groups'
#executable_path = '/Users/eric/Documents/Matematik/Massey-pari/massey'
p = "3"

def run_once(command, timeout=600):
    try:
        r = subprocess.run(command, capture_output=True, text=True,
                           timeout=timeout, check=False, close_fds=True)
        return r.stdout, r.stderr, r.returncode
    except subprocess.TimeoutExpired:
        return None, None, 'TIMEOUT'

infile  = "p_3_all_cyc_[3^m,3^n]_two_missing"
outfile = "IPADs.json"

# Load existing JSON file if it exists
existing_data = []
existing_d_values = set()

if os.path.exists(outfile):
    print(f"Loading existing data from {outfile}...")
    try:
        with open(outfile, 'r') as f:
            existing_data = json.load(f)
        existing_d_values = {entry["D"] for entry in existing_data}
        print(f"Loaded {len(existing_data)} existing entries")
        print(f"Found {len(existing_d_values)} unique D values already in file")
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Warning: Could not load existing file: {e}")
        print("Starting with empty data")
        existing_data = []
        existing_d_values = set()
else:
    print(f"Output file {outfile} does not exist. Starting fresh.")

# Process new entries
new_entries = []
skipped_existing = 0
processed_new = 0

with open(infile, "r") as f:
    for i, line in enumerate(f):
        if i%100 == 0:
            print(i)
        # if i > 100:
        #     break
        D = line.strip()[1:]
        
        # Check if D already exists in the output file
        if D in existing_d_values:
            skipped_existing += 1
            if skipped_existing % 1000 == 0:
                print(f"Skipped {skipped_existing} existing D values...")
            continue
        
        pol = "s^2+" + D
        #print(f"{i} : D: {D}", flush=True)

        stdout, stderr, rc = run_once([executable_path, p, pol], timeout=20)
        if rc == 'TIMEOUT' or stdout is None:
            print(f"Skipped (timeout/runner issue): D={D}")
            continue

        full = stdout
        
        full = stdout
        c = full.find('"C"');  d = full.find('"C"',  c + 3)
        a = full.find('""'); b = full.find('""', a + 2)

        if a == -1 or b == -1 or c == -1 or d == -1:
            print(f"Missing markers, skipping D={D}")
            continue
        
        output_K_cyc = full[c+3:d]
        output = full[a+2:b]

        # Get the p-part of the extension cyc
        # print(output)
        K_cyc = p_part([eval(output_K_cyc)])
        cycs = p_part(eval(output))

        # Create JSON object
        json_entry = {
            "D": D,
            "IPAD": IPAD(K_cyc[0], cycs)
        }
        new_entries.append(json_entry)
        processed_new += 1

# Combine existing and new entries
print(f"\nSummary:")
print(f"  - Existing entries: {len(existing_data)}")
print(f"  - New entries processed: {processed_new}")
print(f"  - Skipped (already existed): {skipped_existing}")
print(f"  - Total entries to write: {len(existing_data) + len(new_entries)}")

# Write combined data to file
print(f"\nWriting to {outfile}...")
all_data = existing_data + new_entries
with open(outfile, 'w') as out:
    json.dump(all_data, out, indent=2, ensure_ascii=False)

print(f"Successfully wrote {len(all_data)} entries to {outfile}")

Loading existing data from gap-files/IPADs-3.json...
Loaded 461923 existing entries
Found 461923 unique D values already in file
0

Summary:
  - Existing entries: 461923
  - New entries processed: 2
  - Skipped (already existed): 0
  - Total entries to write: 461925

Writing to gap-files/IPADs-3.json...
Successfully wrote 461925 entries to gap-files/IPADs-3.json
